## 21.1 클래스 팩토리

In [1]:
class Dog:
    def __init__(self, name, weight, owner):
        self.name = name
        self.weight = weight
        self.owner = owner

In [2]:
rex = Dog('Rex', 30, 'Bob')
rex

In [ ]:
# 예제 21-2: factories.py: 간단한 클래스 팩토리
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError: #replace() or split()을 사용할 수 없다.
        pass
    field_names = tuple(field_names)
    
    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
            
    def __iter__(self):
        for name in self.__slots__:
            yield getattr(self, name)
            
    def __repr__(self):
        values = ','.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)
    
    cls_attrs = dict(__slots__ = field_names,
                    __init__ = __init__,
                    __iter__ = __iter__,
                    __repr__ = __repr__)
    
    return type(cls_name, (object,), cls_attrs)

## 21.4 메타클래스 기본 지식

In [5]:
import collections
collections.Iterable.__class__

/Users/ahrimahn/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


abc.ABCMeta

In [6]:
import abc
abc.ABCMeta.__class__

type

In [7]:
abc.ABCMeta.__mro__

(abc.ABCMeta, type, object)

In [ ]:
# 예제 21-10: evaltime_meta.py: MetaAleph 메타클래스의 객체인 ClassFive
from evalsupport import deco_alpha
from evalsupport import MetaAleph

print('<[1]> evaltime_meta module start')

@deco_alpha
class ClassThree():
    print('<[2]> ClassThree body')
    
    def method_y(self):
        print('<[3]> ClassThree.method_y')
        
class ClassFour(ClassThree):
    print('<[4]> ClassFour body')
    
    def method_y(self):
        print('<[5]> ClassFour.method_y')
        
class ClassFive(metaclass=MetaAleph):
    print('<[6]> Class Five.method_y')
    
    def __init__(self):
        print('<[7]> ClassFive.__init__')
        
    def method_z(self):
        print('<[8]> ClassFive.method_z')
        
class ClassSix(ClassFive):
    print('<[9]> ClassSix body')
    
    def method_z(self):
        print('<[10]> ClassSix.method_z')
        
if __name__ == '__main__':
    print('<[11]> ClassThree tests', 30 * '.')
    three = ClassThree()
    three.method_y()
    print('<[12]> ClassFour tests', 30 * '.')
    four = ClassFour()
    four.method_y()
    print('<[13]> ClassFive tests', 30 * '.')
    five = ClassFive()
    five.method_z()
    print('<[14]> ClassSix tests', 30 * '.')
    six = ClassSix()
    six.method_z
    
print('<[15]> evaltime_meta module end')

In [ ]:
# 예제 21-12: evalsupport.py: [예제 21-7]의 MetaAleph 메타클래스의 정의
class MetaAleph(type):
    print('<[400]> MetaAleph body')
    
    def __init__(cls, name, bases, dic):
        print('<[500]> MetaAleph.__init__')
        
        def inner_2(self):
            print('<[600]> MetaAleph.__init__:inner_2')
            
        cls.method_z = inner_2

In [ ]:
# 예제 21-14: bulkfood_v7.py: 메타클래스가 받쳐주면 model.Entity에서 상속받는 것으로 충분하다.
import model_v7 as model

class LineItem(model.Entity):
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [ ]:
# 예제 21-15 model_v7.py: EntityMeta 메타클래스와 이 메타클래스의 객체인 Entity
class EntityMeta(type):
    """검증된 필드를 가진 비즈니스 개체에 대한 메타클래스"""
    
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        for key, attr in attr_dict.items():
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                
class Entity(metaclass=EntityMeta):
    """검증된 필드를 가진 비즈니스 체계"""

In [ ]:
# 예제 21-16: model_v8.py: __prepare__()를 사용하는 EntityMeta 메타클래스
class EntityMeta(type):
    """검증된 필드를 가진 비즈니스 개체에 대한 메타클래스"""
    
    @classmethod
    def __prepare__(cls, name, bases):
        return collections.OrderedDict()
    
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        cls._field_names = []
        for key, attr in attr_dict.items():
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}',format(type_name, key)
                cls._field_names.append(key)
                
                
class Entity(metaclass=EntityMeta):
    """검증된 필드를 가진 비즈니스 체계"""
    
    @classmethod
    def field_names(cls):
        for name in cls._field_names:
            yield name

## 21.7 객체로서의 클래스

* cls._bases_: 슈퍼클래스들을 담은 튜플
* cls._qualname_: 전역 범위에서 클래스 정의를 담은 모듈까지의 경로를 점으로 구분한, 클래스나 함수의 경로명을 담고 있는 파이썬3.3에 추가된 속성.
* cls._subclasses_(): 이 메서드는 현재 메모리에 존재하는 클래스의 바로 아래 서브클래스들의 리스트를 반환한다.
* cls.mro(): 인터프리터는 이 메서드를 클래스의 _mro_ 속성에 담겨 있는 슈퍼클래스의 튜플을 가져와서 클래스를 생성할 때 호출한다.